In [ ]:
import time, fasttext
fmodel= fasttext.load_model('../data/lid.176.bin')
import pickle as pkl
import matplotlib.pyplot as plt
import tqdm, re, pandas as pd, random, pprint

In [ ]:
def detect(text):
    temp= fmodel.predict([text])
    temp_lang= temp[0][0][0]
    temp_lang= temp_lang[temp_lang.index('__label__')+len('__label__'):]
    return [temp_lang, temp[1][0][0]]

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
# Find the common users
regularBefore= RegB[RegB['user.id_str'].isin(RegA['user.id_str'])].copy()
regularAfter= RegA[RegA['user.id_str'].isin(RegB['user.id_str'])].copy()
# Set the type of user
regularBefore['typeOfUser']= 'regularBefore'
regularAfter['typeOfUser']= 'regularAfter'

In [ ]:
ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
# Find the common users
restoredBefore= ResB[ResB['user.id_str'].isin(ResA['user.id_str'])==True].copy()
restoredAfter= ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()
# Set the type of user
restoredBefore['typeOfUser']= 'restoredBefore'
restoredAfter['typeOfUser']= 'restoredAfter'

In [ ]:
pklDict= {
    'restoredBefore': restoredBefore,
    'restoredAfter': restoredAfter,
    'regularBefore': regularBefore,
    'regularAfter': regularAfter,
}
ResA= None
ResB= None
RegA= None
RegB= None
regularAfter= None
restoredAfter= None
regularBefore= None
restoredBefore= None

In [ ]:
def printLanguageStats(dfName= None, df= None, name= ''):
    temp= None
    print('--'*20)
    if dfName:
        temp= pklDict[dfName][['lang', 'id_str']]
        print(dfName)
    else:
        temp= df
        if len(name)>0:
            print(name)
    print('Total Tweets:', len(temp.index))
    print('No language detected tweets:', len(temp[temp['lang']=='und'].index))
    print('Language detected tweets:', len(temp[temp['lang']!='und'].index))
    print('Number of languages detected:', len(temp[temp['lang']!='und']['lang'].unique()))
    temp_dict= dict(temp[temp['lang']!='und']['lang'].value_counts())
    for i in temp_dict:
        temp_dict[i]= (temp_dict[i]/len(temp[temp['lang']!='und'].index))*100
    temp_dict= {k: v for k, v in sorted(temp_dict.items(), key=lambda item: item[1], reverse= True)}
    print('*Top 5 languages*')
    C= 0
    for i in temp_dict:
        print(i, f'{temp_dict[i]}%')
        C+=1
        if C==5:
            break

In [ ]:
printLanguageStats('restoredBefore')
printLanguageStats('restoredAfter')
printLanguageStats('regularBefore')
printLanguageStats('regularAfter')

In [ ]:
normal_tweets= {'id_str': [], 'lang': [], 'accuracy_lang_detection': []}
normal_dict= {}

with open(f'../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                # If it's a retweet
                if i.get('retweeted_status'):
                    # If it's an extended retweet
                    if i['retweeted_status'].get('extended_tweet'):
                        if i['retweeted_status']['extended_tweet'].get('full_text'):
                            normal_tweets['id_str'].append(i['id_str'])
                            try:
                                tempDetect= detect(i['retweeted_status']['extended_tweet']['full_text'])
                                normal_tweets['lang'].append(tempDetect[0])
                                normal_tweets['accuracy_lang_detection'].append(tempDetect[1])
                            except:
                                normal_tweets['lang'].append('und')
                                normal_tweets['accuracy_lang_detection'].append(0)
                        else:
                            print('--Here--1--')
                            pprint.pprint(i)
                            break
                    
                    # If it's a retweet (non-extended)
                    else:
                        if i['retweeted_status'].get('full_text'):
                            normal_tweets['id_str'].append(i['id_str'])
                            try:
                                tempDetect= detect(i['retweeted_status']['full_text'])
                                normal_tweets['lang'].append(tempDetect[0])
                                normal_tweets['accuracy_lang_detection'].append(tempDetect[1])
                            except:
                                normal_tweets['lang'].append('und')
                                normal_tweets['accuracy_lang_detection'].append(0)
                                
                        elif i['retweeted_status'].get('text'):
                            normal_tweets['id_str'].append(i['id_str'])
                            try:
                                tempDetect= detect(i['retweeted_status']['text'])
                                normal_tweets['lang'].append(tempDetect[0])
                                normal_tweets['accuracy_lang_detection'].append(tempDetect[1])
                            except:
                                normal_tweets['lang'].append('und')
                                normal_tweets['accuracy_lang_detection'].append(0)
                                
                        else:
                            print('--Here--2--')
                            pprint.pprint(i)
                            break
                            
                # If it's an original tweet (extended)
                elif i.get('full_text'):
                    normal_tweets['id_str'].append(i['id_str'])
                    try:
                        tempDetect= detect(i['full_text'])
                        normal_tweets['lang'].append(tempDetect[0])
                        normal_tweets['accuracy_lang_detection'].append(tempDetect[1])
                    except:
                        normal_tweets['lang'].append('und')
                        normal_tweets['accuracy_lang_detection'].append(0)
                        
                # If it's an original tweet (non-extended)
                elif i.get('text'):
                    normal_tweets['id_str'].append(i['id_str'])
                    try:
                        tempDetect= detect(i['text'])
                        normal_tweets['lang'].append(tempDetect[0])
                        normal_tweets['accuracy_lang_detection'].append(tempDetect[1])
                    except:
                        normal_tweets['lang'].append('und')
                        normal_tweets['accuracy_lang_detection'].append(0)
                        
                else:
                    print('--Here--3--')
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
suspended_tweets= {'id_str': [], 'lang': [], 'accuracy_lang_detection': []}
suspended_dict= {}
restored_tweets= {'id_str': [], 'lang': [], 'accuracy_lang_detection': []}
restored_dict= {}

with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status'):
                    if i['retweeted_status'].get('extended_tweet'):
                        if i['retweeted_status']['extended_tweet'].get('full_text'):
                            if i['user']['id_str'] in restoredUsersList:
                                restored_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['extended_tweet']['full_text'])
                                    restored_tweets['lang'].append(tempDetect[0])
                                    restored_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    restored_tweets['lang'].append('und')
                                    restored_tweets['accuracy_lang_detection'].append(0)
                            else:
                                suspended_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['extended_tweet']['full_text'])
                                    suspended_tweets['lang'].append(tempDetect[0])
                                    suspended_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    suspended_tweets['lang'].append('und')
                                    suspended_tweets['accuracy_lang_detection'].append(0)
                        else:
                            print('--Here--1--')
                            pprint.pprint(i)
                            break
                    else:
                        if i['retweeted_status'].get('full_text'):
                            if i['user']['id_str'] in restoredUsersList:
                                restored_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['full_text'])
                                    restored_tweets['lang'].append(tempDetect[0])
                                    restored_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    restored_tweets['lang'].append('und')
                                    restored_tweets['accuracy_lang_detection'].append(0)
                            else:
                                suspended_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['full_text'])
                                    suspended_tweets['lang'].append(tempDetect[0])
                                    suspended_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    suspended_tweets['lang'].append('und')
                                    suspended_tweets['accuracy_lang_detection'].append(0)
                        elif i['retweeted_status'].get('text'):
                            if i['user']['id_str'] in restoredUsersList:
                                restored_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['text'])
                                    restored_tweets['lang'].append(tempDetect[0])
                                    restored_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    restored_tweets['lang'].append('und')
                                    restored_tweets['accuracy_lang_detection'].append(0)
                            else:
                                suspended_tweets['id_str'].append(i['id_str'])
                                try:
                                    tempDetect= detect(i['retweeted_status']['text'])
                                    suspended_tweets['lang'].append(tempDetect[0])
                                    suspended_tweets['accuracy_lang_detection'].append(tempDetect[1])
                                except:
                                    suspended_tweets['lang'].append('und')
                                    suspended_tweets['accuracy_lang_detection'].append(0)
                        else:
                            print('--Here--2--')
                            pprint.pprint(i)
                            break
                elif i.get('full_text'):
                    if i['user']['id_str'] in restoredUsersList:
                        restored_tweets['id_str'].append(i['id_str'])
                        try:
                            tempDetect= detect(i['full_text'])
                            restored_tweets['lang'].append(tempDetect[0])
                            restored_tweets['accuracy_lang_detection'].append(tempDetect[1])
                        except:
                            restored_tweets['lang'].append('und')
                            restored_tweets['accuracy_lang_detection'].append(0)
                    else:
                        suspended_tweets['id_str'].append(i['id_str'])
                        try:
                            tempDetect= detect(i['full_text'])
                            suspended_tweets['lang'].append(tempDetect[0])
                            suspended_tweets['accuracy_lang_detection'].append(tempDetect[1])
                        except:
                            suspended_tweets['lang'].append('und')
                            suspended_tweets['accuracy_lang_detection'].append(0)
                elif i.get('text'):
                    if i['user']['id_str'] in restoredUsersList:
                        restored_tweets['id_str'].append(i['id_str'])
                        try:
                            tempDetect= detect(i['text'])
                            restored_tweets['lang'].append(tempDetect[0])
                            restored_tweets['accuracy_lang_detection'].append(tempDetect[1])
                        except:
                            restored_tweets['lang'].append('und')
                            restored_tweets['accuracy_lang_detection'].append(0)
                    else:
                        suspended_tweets['id_str'].append(i['id_str'])
                        try:
                            tempDetect= detect(i['text'])
                            suspended_tweets['lang'].append(tempDetect[0])
                            suspended_tweets['accuracy_lang_detection'].append(tempDetect[1])
                        except:
                            suspended_tweets['lang'].append('und')
                            suspended_tweets['accuracy_lang_detection'].append(0)
                else:
                    print('--Here--3--')
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
normal_tweets_df= pd.DataFrame.from_dict(normal_tweets).drop_duplicates('id_str')
restored_tweets_df= pd.DataFrame.from_dict(restored_tweets).drop_duplicates('id_str')
suspended_tweets_df= pd.DataFrame.from_dict(suspended_tweets).drop_duplicates('id_str')
printLanguageStats(None, normal_tweets_df, 'Normal')
printLanguageStats(None, restored_tweets_df, 'Restored')
printLanguageStats(None, suspended_tweets_df, 'Suspended')

In [ ]:
import json
normal_dict, suspended_dict, restored_dict= {}, {}, {}
for i, r in tqdm.tqdm(normal_tweets_df.iterrows()):
    normal_dict[r['id_str']]= {
        'lang': r['lang'],
        'accuracy': r['accuracy_lang_detection']
    }
for i, r in tqdm.tqdm(suspended_tweets_df.iterrows()):
    suspended_dict[r['id_str']]= {
        'lang': r['lang'],
        'accuracy': r['accuracy_lang_detection']
    }
for i, r in tqdm.tqdm(restored_tweets_df.iterrows()):
    restored_dict[r['id_str']]= {
        'lang': r['lang'],
        'accuracy': r['accuracy_lang_detection']
    }
with open('./normal_tweets_lang_accuracy.json', 'w') as fp:
    json.dump(normal_dict, fp, indent= 2)
with open('./suspended_tweets_lang_accuracy.json', 'w') as fp:
    json.dump(suspended_dict, fp, indent= 2)
with open('./restored_tweets_lang_accuracy.json', 'w') as fp:
    json.dump(restored_dict, fp, indent= 2)